# Riftfound Metrics Dashboard

Interactive analysis of site traffic and database statistics.

## Setup

Before running this notebook:

1. Download CloudFront logs: `./download-logs.sh 30`
2. Install dependencies: `pip install pandas matplotlib`
3. For database metrics, ensure `deploy.env` is configured (or use local SQLite)

In [ ]:
import gzip
import re
import sqlite3
import subprocess
from collections import defaultdict
from datetime import datetime, timedelta
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

LOGS_DIR = Path('logs')
ROOT_DIR = Path('../..')
LOCAL_DB = ROOT_DIR / 'data' / 'riftfound.db'

## 1. Load CloudFront Logs

In [ ]:
# CloudFront log fields
FIELDS = [
    'date', 'time', 'edge_location', 'bytes', 'client_ip', 'method', 'host',
    'uri_stem', 'status', 'referer', 'user_agent', 'query_string', 'cookie',
    'edge_result_type', 'request_id', 'host_header', 'protocol', 'cs_bytes',
    'time_taken', 'forwarded_for', 'ssl_protocol', 'ssl_cipher',
    'edge_response_result_type', 'protocol_version'
]

def load_cloudfront_logs(logs_dir: Path, days: int = None) -> pd.DataFrame:
    """Load CloudFront logs into a DataFrame."""
    records = []
    cutoff = None
    if days:
        cutoff = (datetime.now() - timedelta(days=days)).strftime('%Y-%m-%d')
    
    for log_file in logs_dir.glob('*'):
        if log_file.suffix == '.gz':
            opener = lambda f: gzip.open(f, 'rt', encoding='utf-8', errors='ignore')
        elif log_file.is_file() and not log_file.name.startswith('.'):
            opener = lambda f: open(f, 'r', encoding='utf-8', errors='ignore')
        else:
            continue
        
        try:
            with opener(log_file) as f:
                for line in f:
                    if line.startswith('#'):
                        continue
                    parts = line.strip().split('\t')
                    if len(parts) >= 10:
                        if cutoff and parts[0] < cutoff:
                            continue
                        record = {FIELDS[i]: parts[i] if i < len(parts) else '' 
                                  for i in range(min(len(FIELDS), len(parts)))}
                        records.append(record)
        except Exception as e:
            print(f"Warning: {log_file}: {e}")
    
    df = pd.DataFrame(records)
    if not df.empty:
        df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
        df['bytes'] = pd.to_numeric(df['bytes'], errors='coerce')
    return df

# Load logs
if LOGS_DIR.exists():
    logs_df = load_cloudfront_logs(LOGS_DIR, days=30)
    print(f"Loaded {len(logs_df):,} log records")
    if not logs_df.empty:
        print(f"Date range: {logs_df['date'].min()} to {logs_df['date'].max()}")
else:
    print("No logs found. Run ./download-logs.sh first.")
    logs_df = pd.DataFrame()

## 2. Traffic Overview

In [ ]:
if not logs_df.empty:
    print("=" * 50)
    print("TRAFFIC SUMMARY")
    print("=" * 50)
    print(f"Total requests:      {len(logs_df):,}")
    print(f"Unique visitors:     {logs_df['client_ip'].nunique():,}")
    print(f"Successful (2xx/3xx): {len(logs_df[logs_df['status'].str.match(r'^[23]')]):,}")
    print(f"Data transferred:    {logs_df['bytes'].sum() / 1e9:.2f} GB")
else:
    print("No log data available")

## 3. Daily Traffic

In [ ]:
if not logs_df.empty:
    daily = logs_df.groupby('date').agg(
        requests=('date', 'count'),
        unique_visitors=('client_ip', 'nunique'),
        bytes_transferred=('bytes', 'sum')
    ).reset_index()
    daily['date'] = pd.to_datetime(daily['date'])
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    # Requests
    axes[0].bar(daily['date'], daily['requests'], color='steelblue', alpha=0.7)
    axes[0].set_ylabel('Requests')
    axes[0].set_title('Daily Requests')
    
    # Unique visitors
    axes[1].bar(daily['date'], daily['unique_visitors'], color='seagreen', alpha=0.7)
    axes[1].set_ylabel('Unique Visitors')
    axes[1].set_title('Daily Unique Visitors (by IP)')
    
    plt.tight_layout()
    plt.show()
    
    # Summary stats
    print(f"\nAverage daily requests: {daily['requests'].mean():,.0f}")
    print(f"Average daily visitors: {daily['unique_visitors'].mean():,.0f}")

## 4. Page Views Analysis

In [ ]:
if not logs_df.empty:
    # Categorize requests
    def categorize_path(path):
        if path in ('/', '/index.html'):
            return 'Homepage'
        elif re.match(r'^/events?/[a-f0-9-]+$', path):
            return 'Event Detail'
        elif path == '/api/events':
            return 'Calendar API'
        elif path.startswith('/api/events/geocode'):
            return 'Location Search'
        elif path.startswith('/api/'):
            return 'Other API'
        elif re.match(r'^/assets/', path) or path.endswith(('.js', '.css', '.png', '.ico')):
            return 'Static Assets'
        else:
            return 'Other'
    
    logs_df['category'] = logs_df['uri_stem'].apply(categorize_path)
    
    # Exclude static assets for page view analysis
    pages_df = logs_df[logs_df['category'] != 'Static Assets']
    
    category_stats = pages_df.groupby('category').agg(
        requests=('category', 'count'),
        unique_visitors=('client_ip', 'nunique')
    ).sort_values('requests', ascending=False)
    
    print("PAGE VIEWS BY CATEGORY")
    print("=" * 50)
    print(category_stats.to_string())
    
    # Pie chart
    fig, ax = plt.subplots(figsize=(10, 8))
    category_stats['requests'].plot.pie(
        autopct='%1.1f%%', 
        ax=ax,
        colors=plt.cm.Set3.colors
    )
    ax.set_ylabel('')
    ax.set_title('Request Distribution by Category')
    plt.show()

## 5. Event Detail Views

In [ ]:
if not logs_df.empty:
    event_views = logs_df[logs_df['category'] == 'Event Detail'].copy()
    
    if not event_views.empty:
        print(f"Total event detail views: {len(event_views):,}")
        print(f"Unique visitors viewing events: {event_views['client_ip'].nunique():,}")
        print(f"Unique events viewed: {event_views['uri_stem'].nunique():,}")
        
        # Daily event views
        daily_events = event_views.groupby('date').size()
        
        plt.figure(figsize=(14, 5))
        daily_events.plot(kind='bar', color='coral', alpha=0.7)
        plt.title('Daily Event Detail Page Views')
        plt.xlabel('Date')
        plt.ylabel('Views')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        
        # Top viewed events
        print("\nTop 10 Most Viewed Events:")
        print(event_views['uri_stem'].value_counts().head(10).to_string())
    else:
        print("No event detail views recorded")

## 6. Geographic Distribution (Edge Locations)

In [ ]:
if not logs_df.empty:
    # CloudFront edge locations indicate approximate user geography
    edge_counts = logs_df['edge_location'].value_counts().head(15)
    
    plt.figure(figsize=(12, 6))
    edge_counts.plot(kind='barh', color='teal', alpha=0.7)
    plt.title('Top 15 CloudFront Edge Locations')
    plt.xlabel('Requests')
    plt.ylabel('Edge Location')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

## 7. Browser Analysis

In [ ]:
if not logs_df.empty:
    def get_browser(ua):
        ua = str(ua).lower()
        if 'bot' in ua or 'crawler' in ua or 'spider' in ua:
            return 'Bot'
        elif 'chrome' in ua and 'edg' not in ua:
            return 'Chrome'
        elif 'firefox' in ua:
            return 'Firefox'
        elif 'safari' in ua and 'chrome' not in ua:
            return 'Safari'
        elif 'edg' in ua:
            return 'Edge'
        else:
            return 'Other'
    
    logs_df['browser'] = logs_df['user_agent'].apply(get_browser)
    browser_counts = logs_df['browser'].value_counts()
    
    fig, ax = plt.subplots(figsize=(10, 8))
    browser_counts.plot.pie(
        autopct='%1.1f%%',
        ax=ax,
        colors=['#4285F4', '#FF7139', '#5FB4CB', '#0078D7', '#888888', '#AAAAAA']
    )
    ax.set_ylabel('')
    ax.set_title('Traffic by Browser')
    plt.show()

---

# Database Metrics

The following cells analyze the SQLite database for event and shop statistics.

In [ ]:
def get_db_connection():
    """Get database connection (local SQLite only)."""
    if LOCAL_DB.exists():
        return sqlite3.connect(LOCAL_DB)
    else:
        print(f"Database not found at {LOCAL_DB}")
        print("For production data, run: ./db-metrics.sh --remote")
        return None

conn = get_db_connection()

## 8. Events & Shops Overview

In [ ]:
if conn:
    overview = pd.read_sql_query("""
        SELECT 'Total Events' as metric, COUNT(*) as value FROM events
        UNION ALL
        SELECT 'Total Shops', COUNT(*) FROM shops
        UNION ALL
        SELECT 'Events with location', COUNT(*) FROM events WHERE shop_id IS NOT NULL
        UNION ALL
        SELECT 'Shops with coordinates', COUNT(*) FROM shops WHERE latitude IS NOT NULL
    """, conn)
    
    print("DATABASE OVERVIEW")
    print("=" * 40)
    for _, row in overview.iterrows():
        print(f"{row['metric']}: {row['value']:,}")

## 9. Events Added Over Time

In [ ]:
if conn:
    events_daily = pd.read_sql_query("""
        SELECT 
            date(created_at) as date,
            COUNT(*) as events_added
        FROM events
        WHERE created_at >= date('now', '-30 days')
        GROUP BY date(created_at)
        ORDER BY date
    """, conn)
    
    if not events_daily.empty:
        events_daily['date'] = pd.to_datetime(events_daily['date'])
        
        plt.figure(figsize=(14, 5))
        plt.bar(events_daily['date'], events_daily['events_added'], color='steelblue', alpha=0.7)
        plt.title('Events Added Per Day (Last 30 Days)')
        plt.xlabel('Date')
        plt.ylabel('Events Added')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        
        print(f"\nTotal events added (30 days): {events_daily['events_added'].sum():,}")
        print(f"Average per day: {events_daily['events_added'].mean():.1f}")
    else:
        print("No events found in the last 30 days")

## 10. Events by Type

In [ ]:
if conn:
    event_types = pd.read_sql_query("""
        SELECT 
            COALESCE(event_type, 'Unknown') as event_type,
            COUNT(*) as count
        FROM events
        GROUP BY event_type
        ORDER BY count DESC
        LIMIT 15
    """, conn)
    
    if not event_types.empty:
        plt.figure(figsize=(12, 6))
        plt.barh(event_types['event_type'], event_types['count'], color='coral', alpha=0.7)
        plt.title('Events by Type')
        plt.xlabel('Count')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()

## 11. Shops by State

In [ ]:
if conn:
    shops_by_state = pd.read_sql_query("""
        SELECT 
            COALESCE(state, 'Unknown') as state,
            COUNT(*) as count
        FROM shops
        WHERE country = 'United States' OR country IS NULL
        GROUP BY state
        ORDER BY count DESC
        LIMIT 20
    """, conn)
    
    if not shops_by_state.empty:
        plt.figure(figsize=(14, 8))
        plt.barh(shops_by_state['state'], shops_by_state['count'], color='seagreen', alpha=0.7)
        plt.title('Shops by State (US)')
        plt.xlabel('Count')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()

## 12. Recent Scrape Runs

In [ ]:
if conn:
    scrape_runs = pd.read_sql_query("""
        SELECT 
            started_at,
            completed_at,
            events_found,
            events_new,
            status
        FROM scrape_runs
        ORDER BY started_at DESC
        LIMIT 15
    """, conn)
    
    if not scrape_runs.empty:
        print("RECENT SCRAPE RUNS")
        print("=" * 80)
        print(scrape_runs.to_string(index=False))
    else:
        print("No scrape runs found")

In [ ]:
# Close database connection
if conn:
    conn.close()
    print("Database connection closed")